In [7]:
import geopandas as gpd
import rasterio
import numpy as np
import random

In [8]:
shp = r'C:\egzamin\data\Lubin_2024_03_27_pc_t1.shp' 
asc = r'C:\egzamin\data\Lubin_2021_06_03.asc'

In [9]:
points = gpd.read_file(shp)

c:\Users\Milosz\miniconda3\envs\exam\lib\site-packages\pyogrio\raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Point' is converted to 'Point Z'
  return ogr_read(


In [18]:
sample_size = min(5000, len(points))  
points_sampled = points.sample(n=sample_size, random_state=42)


In [19]:
with rasterio.open(asc) as src:
    dem_data = src.read(1) 
    dem_transform = src.transform

    coords = [(geom.x, geom.y) for geom in points_sampled.geometry]

    dem_values = [val[0] for val in src.sample(coords)]

In [20]:
points_sampled["DEM_z"] = dem_values

In [21]:
if points_sampled.geometry.iloc[0].has_z:
    points_sampled["Z_diff"] = points_sampled.geometry.apply(lambda p: p.z) - points_sampled["DEM_z"]

In [22]:
deltaH = points_sampled["Z_diff"]
ME = np.mean(deltaH)
MAE = np.mean(np.abs(deltaH))
STD = np.std(deltaH) 
RMSE = np.sqrt(np.mean(deltaH**2)) 

In [25]:
print(f"ME: {ME:.2f} m")
print(f"MAE: {MAE:.2f} m")
print(f"STD: {STD:.2f} m")
print(f"RMSE: {RMSE:.2f} m")

ME: 3.99 m
MAE: 4.30 m
STD: 8.52 m
RMSE: 9.41 m
